In [1]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=68585 sha256=fa53067eb718c08bed9919e55cbc17d83de72e69291a31deb339550e7ad56e00
  Stored in directory: /tmp/pip-ephem-wheel-cache-uz5i0cor/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY
transform = T.Compose([T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
    transform=transform
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test",
    transform=transform
)
testset.initialize()

In [4]:
from spuco.models import model_factory 
from torch.optim import SGD
from spuco.end2end import LFF

bias_model=model_factory("lenet", trainset[0][0].shape, len(classes)).to(device)
model=model_factory("lenet", trainset[0][0].shape, len(classes)).to(device)
lff = LFF(
    trainset=trainset,
    bias_model=bias_model,
    debias_model=model,
    batch_size=4,
    bias_optimizer=SGD(bias_model.parameters(), lr=1e-3, weight_decay=5e-4),
    debias_optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4),
    q=0.2,
    num_epochs=1,
    device=device,
    verbose=True
)
lff.train()

Epoch 0:   0%|          | 0/12001 [00:00<?, ?batch/s]

tensor([[-0.0383, -0.0377, -0.0231, -0.0357, -0.0251],
        [-0.0436, -0.0405, -0.0321, -0.0108, -0.0496],
        [-0.0220, -0.0245, -0.0208, -0.0486, -0.0208],
        [-0.0315, -0.0343, -0.0367, -0.0441, -0.0182]], device='cuda:7',
       grad_fn=<AddmmBackward0>)
tensor([[0.2489, 0.2491, 0.2513, 0.2498, 0.2508],
        [0.2476, 0.2484, 0.2490, 0.2560, 0.2447],
        [0.2530, 0.2524, 0.2518, 0.2465, 0.2519],
        [0.2506, 0.2500, 0.2479, 0.2477, 0.2526]], device='cuda:7',
       grad_fn=<SoftmaxBackward0>)
tensor([0.7584, 0.7573, 0.7596, 0.7594], device='cuda:7', dtype=torch.float64)


Epoch 0:   0%|          | 9/12001 [00:01<21:48,  9.16batch/s, bias_accuracy=25.0%, bias_loss=1.22, debias_accuracy=25.0%, debias_loss=0.806]  

tensor([[-4.6636e-02, -3.8617e-02, -2.9326e-02, -1.3396e-02, -2.8130e-02],
        [-3.5797e-02, -4.6069e-02, -8.5978e-05, -1.8320e-02, -2.6016e-02],
        [-2.2753e-02, -2.8907e-02, -2.5706e-02, -2.4391e-02, -2.4244e-02],
        [-4.1555e-02, -3.1764e-02, -3.1684e-02, -2.9457e-02, -1.6896e-02]],
       device='cuda:7', grad_fn=<AddmmBackward0>)
tensor([[0.2475, 0.2494, 0.2481, 0.2520, 0.2489],
        [0.2502, 0.2476, 0.2554, 0.2508, 0.2494],
        [0.2535, 0.2519, 0.2490, 0.2492, 0.2499],
        [0.2488, 0.2511, 0.2475, 0.2480, 0.2517]], device='cuda:7',
       grad_fn=<SoftmaxBackward0>)
tensor([0.7567, 0.7564, 0.7578, 0.7589], device='cuda:7', dtype=torch.float64)
tensor([[-0.0389, -0.0380, -0.0403, -0.0203, -0.0351],
        [-0.0456, -0.0408, -0.0339, -0.0229, -0.0329],
        [-0.0511, -0.0437, -0.0028, -0.0315, -0.0191],
        [-0.0447, -0.0384, -0.0330, -0.0106, -0.0381]], device='cuda:7',
       grad_fn=<AddmmBackward0>)
tensor([[0.2516, 0.2506, 0.2468, 0.2502, 0.249

Epoch 0:   0%|          | 28/12001 [00:01<06:16, 31.84batch/s, bias_accuracy=75.0%, bias_loss=1.21, debias_accuracy=0.0%, debias_loss=0.808] 

...

Epoch 0:   9%|▉         | 1055/12001 [00:12<01:52, 97.64batch/s, bias_accuracy=50.0%, bias_loss=1.11, debias_accuracy=50.0%, debias_loss=0.75]   

tensor([[ 0.0736, -0.0108, -0.0802, -0.0966, -0.0253],
        [-0.1201, -0.0908,  0.0190,  0.1100, -0.1138],
        [-0.1712, -0.1559, -0.0135,  0.2929, -0.1672],
        [ 0.0631, -0.0212, -0.0683, -0.0733, -0.0116]], device='cuda:7',
       grad_fn=<AddmmBackward0>)
tensor([[0.2781, 0.2647, 0.2389, 0.2114, 0.2634],
        [0.2291, 0.2444, 0.2638, 0.2600, 0.2411],
        [0.2177, 0.2290, 0.2554, 0.3121, 0.2285],
        [0.2752, 0.2620, 0.2418, 0.2164, 0.2670]], device='cuda:7',
       grad_fn=<SoftmaxBackward0>)
tensor([0.7742, 0.7661, 0.7923, 0.7725], device='cuda:7', dtype=torch.float64)
tensor([[-0.1266, -0.0825, -0.0121,  0.1264, -0.1241],
        [-0.1611, -0.1342, -0.0162,  0.2129, -0.1479],
        [-0.0149, -0.0523, -0.0774,  0.0158, -0.0250],
        [ 0.0030, -0.0553, -0.0933, -0.0130, -0.0027]], device='cuda:7',
       grad_fn=<AddmmBackward0>)
tensor([[0.2368, 0.2495, 0.2594, 0.2594, 0.2376],
        [0.2288, 0.2369, 0.2584, 0.2828, 0.2319],
        [0.2648, 0.2572, 0

Epoch 0:   9%|▉         | 1079/12001 [00:12<02:06, 86.66batch/s, bias_accuracy=50.0%, bias_loss=1.15, debias_accuracy=50.0%, debias_loss=0.766] 

tensor([[ 0.0855, -0.0047, -0.0664, -0.0897, -0.0113],
        [ 0.0871, -0.0179, -0.0708, -0.0887, -0.0092],
        [-0.0227, -0.0198, -0.0264, -0.0092, -0.0571],
        [-0.0241, -0.0228, -0.0400, -0.0146, -0.0691]], device='cuda:7',
       grad_fn=<AddmmBackward0>)
tensor([[0.2635, 0.2529, 0.2461, 0.2402, 0.2563],
        [0.2639, 0.2496, 0.2450, 0.2405, 0.2569],
        [0.2365, 0.2491, 0.2562, 0.2604, 0.2449],
        [0.2361, 0.2484, 0.2527, 0.2590, 0.2419]], device='cuda:7',
       grad_fn=<SoftmaxBackward0>)
tensor([0.7659, 0.7661, 0.7573, 0.7569], device='cuda:7', dtype=torch.float64)
tensor([[-0.1221, -0.0746,  0.0065,  0.1281, -0.1520],
        [ 0.0080, -0.0615, -0.0614, -0.0014, -0.0176],
        [-0.2034, -0.1679, -0.0210,  0.3303, -0.2029],
        [-0.0305, -0.0168, -0.0429, -0.0219, -0.0643]], device='cuda:7',
       grad_fn=<AddmmBackward0>)
tensor([[0.2406, 0.2510, 0.2591, 0.2523, 0.2389],
        [0.2740, 0.2543, 0.2421, 0.2217, 0.2733],
        [0.2218, 0.2287, 0

KeyboardInterrupt: 

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=bias_model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()

In [8]:
import torch
from torch import nn 

a = torch.tensor([[1., 2.], [2., 4.]])
torch.softmax(a, dim=0)

tensor([[0.2689, 0.1192],
        [0.7311, 0.8808]])